In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
tf.config.run_functions_eagerly(False)

from tensorflow.keras.optimizers.schedules import CosineDecay
from sklearn.model_selection import GroupShuffleSplit
from tensorflow.keras import layers, models, losses, metrics
from tensorflow.keras import regularizers, callbacks
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LeakyReLU, Reshape, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Activation, Add, Input, GlobalAveragePooling1D, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanAbsoluteError
from tensorflow.keras.utils import plot_model

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def add_remaining_useful_life(df):
    # Get the total number of cycles for each unit
    grouped_by_unit = df.groupby(by="unit_nr")
    max_cycle = grouped_by_unit["time_cycles"].max()

    # Merge the max cycle back into the original frame
    result_frame = df.merge(max_cycle.to_frame(name='max_cycle'), left_on='unit_nr', right_index=True)

    # Calculate remaining useful life for each row
    remaining_useful_life = result_frame["max_cycle"] - result_frame["time_cycles"]
    result_frame["RUL"] = remaining_useful_life

    # Drop max_cycle as it's no longer needed
    result_frame = result_frame.drop("max_cycle", axis=1)
    return result_frame

def add_operating_condition(df):
    df_op_cond = df.copy()

    df_op_cond['setting_1'] = abs(df_op_cond['setting_1'].round())
    df_op_cond['setting_2'] = abs(df_op_cond['setting_2'].round(decimals=2))

    # Converting settings to string and concatanating makes the operating condition into a categorical variable
    df_op_cond['op_cond'] = df_op_cond['setting_1'].astype(str) + '_' + \
                        df_op_cond['setting_2'].astype(str) + '_' + \
                        df_op_cond['setting_3'].astype(str)

    return df_op_cond

def condition_scaler(df_train, df_test, sensor_names):
    # Apply operating condition specific scaling
    scaler = StandardScaler()
    for condition in df_train['op_cond'].unique():
        scaler.fit(df_train.loc[df_train['op_cond']==condition, sensor_names])
        df_train.loc[df_train['op_cond']==condition, sensor_names] = scaler.transform(df_train.loc[df_train['op_cond']==condition, sensor_names])
        df_test.loc[df_test['op_cond']==condition, sensor_names] = scaler.transform(df_test.loc[df_test['op_cond']==condition, sensor_names])
    return df_train, df_test

def exponential_smoothing(df, sensors, n_samples, alpha=0.4):
    df = df.copy()
    # Take the exponential weighted mean
    df[sensors] = df.groupby('unit_nr')[sensors].apply(lambda x: x.ewm(alpha=alpha).mean()).reset_index(level=0, drop=True)

    # Drop first n_samples of each unit_nr to reduce filter delay
    def create_mask(data, samples):
        result = np.ones_like(data)
        result[0:samples] = 0
        return result

    mask = df.groupby('unit_nr')['unit_nr'].transform(create_mask, samples=n_samples).astype(bool)
    df = df[mask]

    return df

def gen_train_data(df, sequence_length, columns):
    data = df[columns].values
    num_elements = data.shape[0]

    for start, stop in zip(range(0, num_elements-(sequence_length-1)), range(sequence_length, num_elements+1)):
        yield data[start:stop, :]

def gen_data_wrapper(df, sequence_length, columns, unit_nrs=np.array([])):
    if unit_nrs.size <= 0:
        unit_nrs = df['unit_nr'].unique()

    data_gen = (list(gen_train_data(df[df['unit_nr']==unit_nr], sequence_length, columns))
               for unit_nr in unit_nrs)
    data_array = np.concatenate(list(data_gen)).astype(np.float32)
    return data_array

def gen_labels(df, sequence_length, label):
    data_matrix = df[label].values
    num_elements = data_matrix.shape[0]

    return data_matrix[sequence_length-1:num_elements, :]

def gen_label_wrapper(df, sequence_length, label, unit_nrs=np.array([])):
    if unit_nrs.size <= 0:
        unit_nrs = df['unit_nr'].unique()

    label_gen = [gen_labels(df[df['unit_nr']==unit_nr], sequence_length, label)
                for unit_nr in unit_nrs]
    label_array = np.concatenate(label_gen).astype(np.float32)
    return label_array

def gen_test_data(df, sequence_length, columns, mask_value):
    if df.shape[0] < sequence_length:
        data_matrix = np.full(shape=(sequence_length, len(columns)), fill_value=mask_value)
        idx = data_matrix.shape[0] - df.shape[0]
        data_matrix[idx:,:] = df[columns].values
    else:
        data_matrix = df[columns].values

    stop = data_matrix.shape[0]
    start = stop - sequence_length
    for i in list(range(1)):
        yield data_matrix[start:stop, :]


def get_data(dataset, sensors, sequence_length, alpha, threshold):
	dir_path = './CMAPSS/'
	train_file = 'train_'+dataset+'.txt'
	test_file = 'test_'+dataset+'.txt'

	index_names = ['unit_nr', 'time_cycles']
	setting_names = ['setting_1', 'setting_2', 'setting_3']
	sensor_names = ['s_{}'.format(i+1) for i in range(0,21)]
	col_names = index_names + setting_names + sensor_names

	train = pd.read_csv((dir_path+train_file), sep=r'\s+', header=None,
					 names=col_names)
	test = pd.read_csv((dir_path+test_file), sep=r'\s+', header=None,
					 names=col_names)
	y_test = pd.read_csv((dir_path+'RUL_'+dataset+'.txt'), sep=r'\s+', header=None,
					 names=['RemainingUsefulLife'])

	train = add_remaining_useful_life(train)
	train['RUL'].clip(upper=threshold, inplace=True)

  #Dropping sensors
	drop_sensors = [element for element in sensor_names if element not in sensors]

  # Scale with respect to the operating condition
	X_train_pre = add_operating_condition(train.drop(drop_sensors, axis=1))
	X_test_pre = add_operating_condition(test.drop(drop_sensors, axis=1))
	X_train_pre, X_test_pre = condition_scaler(X_train_pre, X_test_pre, sensors)

  # Exponential smoothing
	X_train_pre= exponential_smoothing(X_train_pre, sensors, 0, alpha)
	X_test_pre = exponential_smoothing(X_test_pre, sensors, 0, alpha)

  # Train/Validation split
	gss = GroupShuffleSplit(n_splits=1, train_size=0.80, random_state=42)

	for train_unit, val_unit in gss.split(X_train_pre['unit_nr'].unique(), groups=X_train_pre['unit_nr'].unique()):
		train_unit = X_train_pre['unit_nr'].unique()[train_unit]  # gss returns indexes and index starts at 1
		val_unit = X_train_pre['unit_nr'].unique()[val_unit]

		x_train = gen_data_wrapper(X_train_pre, sequence_length, sensors, train_unit)
		y_train = gen_label_wrapper(X_train_pre, sequence_length, ['RUL'], train_unit)

		x_val = gen_data_wrapper(X_train_pre, sequence_length, sensors, val_unit)
		y_val = gen_label_wrapper(X_train_pre, sequence_length, ['RUL'], val_unit)

	# Create sequences for test
	test_gen = (list(gen_test_data(X_test_pre[X_test_pre['unit_nr']==unit_nr], sequence_length, sensors, -99.))
			   for unit_nr in X_test_pre['unit_nr'].unique())
	x_test = np.concatenate(list(test_gen)).astype(np.float32)
	test_unit_ids = X_test_pre['unit_nr'].unique()

	return x_train, y_train, x_val, y_val, x_test, y_test['RemainingUsefulLife'], test_unit_ids

In [6]:
# Choose the subset (FD001, FD002, FD003, FD004)
dataset = 'FD002'

# Sensors to use; sensor 13 is dropped from FD002 and FD004
if(dataset == 'FD001' or 'FD003'):
  sensors = ['s_2', 's_3', 's_4', 's_7', 's_8', 's_9', 's_11', 's_12', 's_13', 's_14', 's_15', 's_17', 's_20', 's_21']
else:
  sensors = ['s_2', 's_3', 's_4', 's_7', 's_8', 's_9', 's_11', 's_12', 's_14', 's_15', 's_17', 's_20', 's_21']

sequence_length = 30
alpha = 0.3
rul_clip_threshold = 125

# Load and process the data
x_train, y_train, x_val, y_val, x_test, y_test, test_unit_ids = get_data(
    dataset=dataset,
    sensors=sensors,
    sequence_length=sequence_length,
    alpha=alpha,
    threshold=rul_clip_threshold
)

<ipython-input-4-edafd160830c>:118: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['RUL'].clip(upper=threshold, inplace=True)
<ipython-input-4-edafd160830c>:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[-0.1674041  -1.59188977 -0.1674041  ...  0.54483874  1.96932442
  2.68156725]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train['op_cond']==condition, sensor_names] = scaler.

In [7]:
# Positional Encoding
class PositionalEncodingLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(PositionalEncodingLayer, self).__init__(**kwargs)

    def call(self, x):
        seq_len = tf.shape(x)[1]
        d_model = tf.shape(x)[2]
        pos = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
        i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]
        angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        angle_rads = pos * angle_rates
        sines = tf.sin(angle_rads[:, 0::2])
        cosines = tf.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sines, cosines], axis=-1)
        return x + pos_encoding[tf.newaxis, :, :]

    def get_config(self):
        config = super(PositionalEncodingLayer, self).get_config()
        return config

# CBAM Block
def cbam_block(x, reduction_ratio=8):
    input_channels = x.shape[-1]

    # Channel Attention
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)

    shared_dense_1 = layers.Dense(input_channels // reduction_ratio, activation='relu')
    shared_dense_2 = layers.Dense(input_channels)

    avg_out = shared_dense_2(shared_dense_1(avg_pool))
    max_out = shared_dense_2(shared_dense_1(max_pool))

    channel_attention = layers.Add()([avg_out, max_out])
    channel_attention = layers.Activation('sigmoid')(channel_attention)
    channel_attention = layers.Reshape((1, input_channels))(channel_attention)

    x = layers.Multiply()([x, channel_attention])

    # Spatial Attention
    avg_pool_spatial = layers.Lambda(lambda z: tf.reduce_mean(z, axis=-1, keepdims=True))(x)
    max_pool_spatial = layers.Lambda(lambda z: tf.reduce_max(z, axis=-1, keepdims=True))(x)
    concat = layers.Concatenate(axis=-1)([avg_pool_spatial, max_pool_spatial])
    spatial_attention = layers.Conv1D(1, kernel_size=7, padding='same', activation='sigmoid')(concat)

    x = layers.Multiply()([x, spatial_attention])
    return x


# Residual Dilated CNN Block
def residual_dilated_conv_block(x, filters, kernel_size, dilation_rate):
    shortcut = x
    x = layers.Conv1D(filters, kernel_size, padding='same', dilation_rate=dilation_rate)(x)
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv1D(filters, kernel_size, padding='same', dilation_rate=dilation_rate)(x)
    x = layers.LayerNormalization()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Add()([shortcut, x]) if shortcut.shape[-1] == x.shape[-1] else layers.Conv1D(filters, 1)(shortcut) + x
    x = layers.Activation('relu')(x)
    return x

# Transformer Block (MultiHead Attention)
def transformer_block(x, num_heads=4, ff_dim=128):
    attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(x, x)
    attn_output = layers.Dropout(0.3)(attn_output)
    out1 = layers.LayerNormalization()(x + attn_output)
    ffn = layers.Dense(ff_dim, activation='relu')(out1)
    ffn = layers.Dense(x.shape[-1])(ffn)
    ffn = layers.Dropout(0.3)(ffn)
    return layers.LayerNormalization()(out1 + ffn)

# Model Definition
def create_bilstm_cnn_hybrid_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = PositionalEncodingLayer()(inputs)

    # BiLSTM Stack
    x = layers.Bidirectional(layers.LSTM(96, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(48, return_sequences=True))(x)

    # Transformer Block
    x = transformer_block(x)

    # CNN Blocks
    x = residual_dilated_conv_block(x, filters=64, kernel_size=5, dilation_rate=1)
    x = residual_dilated_conv_block(x, filters=32, kernel_size=5, dilation_rate=2)
    x = cbam_block(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(1e-5))(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs, outputs)
    return model


In [1]:
!pip install -q -U keras-tuner
import keras_tuner as kt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
def build_model(hp):
    input_shape = (x_train.shape[1], x_train.shape[2])
    inputs = layers.Input(shape=input_shape)
    x = PositionalEncodingLayer()(inputs)

    # BiLSTM Stack
    x = layers.Bidirectional(layers.LSTM(hp.Int('lstm_units1', 64, 128, step=32), return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(hp.Int('lstm_units2', 32, 64, step=16), return_sequences=True))(x)

    # Transformer Block
    x = transformer_block(
        x,
        num_heads=hp.Choice('num_heads', [2, 4, 8]),
        ff_dim=hp.Choice('ff_dim', [64, 128, 256])
    )

    # Residual Dilated Convolutions
    x = residual_dilated_conv_block(
        x,
        filters=hp.Choice('res_filters', [32, 64, 128]),
        kernel_size=hp.Choice('kernel_size', [3, 5]),
        dilation_rate=1
    )
    x = residual_dilated_conv_block(
        x,
        filters=hp.Choice('res_filters_2', [32, 64, 128]),
        kernel_size=hp.Choice('kernel_size_2', [3, 5]),
        dilation_rate=2
    )

    # CBAM
    x = cbam_block(x)

    # Output
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(
        hp.Choice('dense_units', [32, 64, 128]),
        activation='relu',
        kernel_regularizer=regularizers.l2(1e-5)
    )(x)
    x = layers.Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1))(x)
    outputs = layers.Dense(1)(x)

    model = models.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.AdamW(
            learning_rate=hp.Float('lr', 1e-4, 1e-2, sampling='log'),
            weight_decay=hp.Choice('weight_decay', [1e-6, 1e-5, 1e-4])
        ),
        loss=tf.keras.losses.Huber(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

    return model


In [ ]:
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_root_mean_squared_error',
    max_trials=20,
    directory='tuner_results',
    project_name='bilstm_cnn_transformer_cbam'
)

tuner.search(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=64,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)]
)

best_model = tuner.get_best_models(1)[0]
best_hps = tuner.get_best_hyperparameters(1)[0]

# Predict on the test set
predictions = best_model.predict(x_test).flatten()

# Evaluate metrics
rmse = np.sqrt(np.mean((predictions - y_test) ** 2))
mae = np.mean(np.abs(predictions - y_test))

print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE: {mae:.4f}")


Trial 20 Complete [00h 03m 31s]
val_root_mean_squared_error: 18.657495498657227

Best val_root_mean_squared_error So Far: 14.860320091247559
Total elapsed time: 01h 13m 48s


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 118 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step
Test RMSE: 25.3737
Test MAE: 17.2746


In [ ]:
print("Best Hyperparameters:")
for param in best_hps.values:
    print(f"{param}: {best_hps.get(param)}")

Best Hyperparameters:
lstm_units1: 96
lstm_units2: 48
num_heads: 4
ff_dim: 128
res_filters: 64
kernel_size: 5
res_filters_2: 32
kernel_size_2: 5
dense_units: 64
dropout: 0.30000000000000004
lr: 0.0005668473081877774
weight_decay: 1e-05


In [ ]:
all_trials = tuner.oracle.trials.values()

# Build a list of dictionaries with hyperparameters and scores
results = []
for trial in all_trials:
    trial_data = trial.hyperparameters.values.copy()
    trial_data['score'] = trial.score  # Typically validation loss or whatever your objective is
    results.append(trial_data)

# Create a DataFrame
df = pd.DataFrame(results)

# Save to CSV
df.to_csv("keras_tuner_trials.csv", index=False)
print("Saved all trial results to keras_tuner_trials.csv")

Saved all trial results to keras_tuner_trials.csv
